In [1]:
import numpy as np
import matplotlib.pyplot as plt
import random
import math 
import pandas as pd

In [2]:
def random_val():
    val = random.random()
    num = 0
    if val>0.5:
        num = 1
    else:
        num = -1
    return num


def randomPath(maxTime,minTimePP,maxTimePP,maxDistance,lastCoor,sat_x,sat_y):
    #maxTime = maximum time of one execution.
    #minTimePP = minimum time between two points of the path.
    #maxTimePP = maximum time between two points of the path.
    #maxDistance = maximum distance between two points of the path.
    #sat_x = maximum coordenate in x.
    #sat_y = maximum coordenate in y.

    time_array = np.array([],dtype=np.int32)
    coor_path = np.array([lastCoor],dtype=np.int32).reshape((1,-1))
    flag = False
    while not flag:
        num = int(random.uniform(minTimePP,maxTimePP))
        if time_array.sum()+num >= maxTime:
            flag = True 
        else:
            time_array = np.append(time_array,num)
    for i in range(len(time_array)):
        if i == 0:
            continue
        else:
            val_x = int(random.uniform(coor_path[i-1,0],coor_path[i-1,0]+maxDistance))
            val_y = int(random.uniform(coor_path[i-1,1],coor_path[i-1,1]+maxDistance))
            val_z = coor_path[0,2]

            if abs(val_x) > sat_x:
                val_x = math.copysign(sat_x,val_x)

            if abs(val_y) > sat_y:
                val_y = math.copysign(sat_y,val_y)

            rand_x = random_val()
            rand_y = random_val()

            coor_path = np.append(coor_path,[[rand_x*val_x,rand_y*val_y,val_z]],axis=0)
    return time_array,coor_path

[ 89  80  60  73 116  81  56 119]


In [4]:
def cal_Total_Distance(pathArray):
    distance_Total = 0
    distance_Array = np.zeros(pathArray.shape[0])
    for i in range(1,pathArray.shape[0]):
        distance_Array[i-1] = np.linalg.norm(pathArray[i-1]-pathArray[i])
    distance_Total = distance_Array.sum()
        
    return distance_Total,distance_Array

def cal_Total_Turns(pathArray):
    total_Turns = 0
    flag_turn = np.zeros(pathArray.shape[0]-1)
    for i in range(1,pathArray.shape[0]-1):
        dot_val = np.dot(pathArray[i-1]-pathArray[i],pathArray[i+1]-pathArray[i])
        mag_A = np.linalg.norm(pathArray[i-1]-pathArray[i])
        mag_B = np.linalg.norm(pathArray[i+1]-pathArray[i])
        fracc = dot_val/(np.dot(mag_A,mag_B)+1E-9)
        #print(fracc)
        flag = math.acos(fracc)
        flag_turn[i-1] = flag
        if flag != 0.0:
            total_Turns+=1
    return total_Turns


In [5]:
def changeToTxt(pathArr,timeArr,idx):
    time = ''
    path = ''
    for i in timeArr:
        time =  time + str(i) + ','

    for i in pathArr:
        path = path + str(i) + ';'

    final = str(idx)+'|'+path+'|'+time
    final = final[:-1]
    return final

1|[5 4 3];[ 5 -5  3];[-5  4  3];[ 3 -4  3];[-4  1  3];[ 1 -2  3];[-1  0  3];[0 0 3];|89,80,60,73,116,81,56,119


In [7]:
##### Ubuntu
#file_path = '/home/sebastian/Desktop/'
#file = open(file_path + 'path_v1.txt',mode="w")

##### Windows
file_path = "C:/Users/SEBASTIAN/OneDrive - Universidad de los Andes/OctavoSemestre/ProblemaEspecial/ML/path/"
file = open(file_path + 'path_v4.txt',mode="w")
file_pd = open(file_path + 'distanceTurn_v4.csv',mode="w")

file_pd.close()
numRoad = 2000
disTotalArray = np.zeros((numRoad,1))
totalTurnsArray = np.zeros((numRoad,1))

for i in range(numRoad):
    if i == 0:
        time,path = randomPath(700,50,120,1,[5,4,1],5,5)
        disTotal,disArray = cal_Total_Distance(path)
        total_turns = cal_Total_Turns(path)
    else:
        time,path = randomPath(700,50,120,3,path[-1,:],5,5)
        disTotal,disArray = cal_Total_Distance(path)
        total_turns = cal_Total_Turns(path)

    disTotalArray[i,0] = disTotal
    totalTurnsArray[i,0] = total_turns

    path = path.astype(int)
    final = ''
    final = changeToTxt(path,time,i+1)
    file.write('\n')
    file.write(final)

file.close()

d = {'disTotal':disTotalArray.reshape(-1),'totalTurns':totalTurnsArray.reshape(-1)}
pd_values = pd.DataFrame(data=d)
pd_values.to_csv(file_path + 'distanceTurn_v4.csv',index=False)


In [48]:
d = {'disTotal':disTotalArray.reshape(-1),'totalTurns':totalTurnsArray.reshape(-1)}
pd_values = pd.DataFrame(data=d)
print(pd_values.head())

    disTotal  totalTurns
0  49.480554         7.0
1  26.280507         7.0
2  27.383326         7.0
3  13.892922         6.0
4  45.215317         7.0


In [49]:
pd_values.to_csv(file_path + 'distanceTurn.csv',index=False)